## Import code

In [2]:
%load_ext autoreload
%autoreload 2 
# Autoreloads packages when changes are detected

from src.Data import * # Class containing the data
from src.Assignment import * # Class containing an assignment
from src.Model import * # Class containing a Pulp model used for optimization
from src.ModelColumnGen import * # Class containing Pulp model that optimizes using column generation
from src.ModelFracStable import * # Class containing a Pulp model for finding an fractionally stable stochastic improvement
from src.ModelHeuristicLP import * # Class containing heuristic that will use generated weakly stable matchings as an input
from src.DataGen import * # Generate student preferences and school priorities
from src.DA_STB import * # Generate DA assignment with single tie-breaking (STB)

## Column generation formulation

In [3]:
# Generate random data
parameters = DataGenParam(mean_pref = 4, capacity_ratio = 1) # Default parameters, except for mean_pref and capacity_ratio
#MyData = generate_data(n_students=400, n_schools=20, parameters = parameters, name="Test_DataGen", print_data=False, seed = 15)
MyData = generate_data(n_students=30, n_schools=6, parameters = parameters, name="500_25", print_data=False, seed = 1)

In [4]:
# Print data if desired
print(MyData)

The data instance has the following properties: 

	30 students.
	6 schools. 

 	PREFERENCES:
	0	1 5 3 
	1	3 4 1 
	2	5 3 4 
	3	1 5 2 0 
	4	5 4 3 1 
	5	1 0 4 
	6	4 5 1 
	7	1 4 5 
	8	5 1 3 4 
	9	2 4 0 
	10	4 5 3 2 1 
	11	1 5 2 4 0 
	12	5 
	13	1 3 
	14	2 4 1 
	15	2 0 1 
	16	2 0 4 3 
	17	4 3 1 5 
	18	3 5 1 2 0 4 
	19	1 2 
	20	0 2 5 
	21	4 1 2 3 5 0 
	22	3 5 2 1 
	23	2 3 0 1 
	24	1 2 3 
	25	3 5 
	26	4 5 3 1 
	27	5 3 2 0 
	28	3 1 
	29	1 3 5 


 	CAPACITIES & PRIORITIES:
	0	5	{27 18 12 13 7 8 29 23 28 9} {16 3 17 19 4 24 2 25 21 20} {14 0 6 26 1 22 11 5 15 10} 
	1	8	{6 5 17 21 18 16 3 25 8 9} {11 23 29 27 12 0 26 10 7 20} {24 19 28 15 14 1 4 13 22 2} 
	2	5	{15 27 19 9 18 17 8 6 10 20} {0 5 23 29 21 13 16 7 24 14} {26 25 1 28 11 3 4 22 2 12} 
	3	1	{24 4 16 28 22 9 5 0 3 13} {27 6 15 7 11 12 25 19 1 2} {23 17 8 14 18 10 29 20 26 21} 
	4	7	{4 19 9 13 7 8 11 10 18 1} {2 29 20 27 24 3 23 5 17 15} {16 25 26 12 14 0 6 28 22 21} 
	5	6	{16 0 10 9 13 15 27 23 25 28} {7 11 14 21 19 22 4 29 1 6} {5 2 24 2

In [5]:
# Generate the assignment from DA with Single Tie-Breaking with n_iter samples
n_iter = 1000
A = DA_STB(MyData, n_iter, 0, True)
print(A.assignment)

Students in ties: 30
Tie-breaking rules needed: 265252859812191058636308480000000
Tie-breaking rules sampled: 1000


100%|██████████| 1000/1000 [00:01<00:00, 698.98it/s]


[[0.    1.    0.    0.    0.    0.   ]
 [0.    0.    0.    0.    1.    0.   ]
 [0.    0.    0.    0.    0.395 0.605]
 [0.    1.    0.    0.    0.    0.   ]
 [0.    0.    0.    0.    0.    1.   ]
 [0.    1.    0.    0.    0.    0.   ]
 [0.    0.    0.    0.    0.985 0.015]
 [0.    1.    0.    0.    0.    0.   ]
 [0.    0.382 0.    0.    0.    0.618]
 [0.    0.    1.    0.    0.    0.   ]
 [0.    0.    0.    0.    1.    0.   ]
 [0.    1.    0.    0.    0.    0.   ]
 [0.    0.    0.    0.    0.    0.597]
 [0.    0.364 0.    0.123 0.    0.   ]
 [0.    0.002 0.716 0.    0.226 0.   ]
 [0.    0.    1.    0.    0.    0.   ]
 [0.325 0.    0.675 0.    0.    0.   ]
 [0.    0.    0.    0.    1.    0.   ]
 [0.    0.389 0.    0.    0.    0.611]
 [0.    0.364 0.636 0.    0.    0.   ]
 [1.    0.    0.    0.    0.    0.   ]
 [0.    0.016 0.    0.    0.984 0.   ]
 [0.    0.    0.    0.446 0.    0.554]
 [0.322 0.    0.678 0.    0.    0.   ]
 [0.    0.377 0.295 0.013 0.    0.   ]
 [0.    0.    0.    0.   

In [ ]:
# Solve the formulations
    # 'IMPR_RANK' refers to minimizing the expected rank while ensuring ex-post stability
MyModel = ModelColumnGen(MyData, A, True)
q = MyModel.Solve("TRAD", "GUROBI", True)
#q = MyModel.Solve("STABLE", "GUROBI", True)

219
[[0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]]
<class 'numpy.ndarray'>
[[0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 

c:\Users\tdemeule\AppData\Local\anaconda3\Lib\site-packages\pulp\pulp.py:1298: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Cutting planes:
  Gomory: 4
  Cover: 3
  MIR: 4
  StrongCG: 1
  Zero half: 3
  RLT: 3

Explored 1 nodes (50 simplex iterations) in 0.07 seconds (0.01 work units)
Thread count was 8 (of 8 available processors)

Solution count 5: -1.36667 -1.23333 -1.2 ... -1.1
No other solutions better than -1.36667

Optimal solution found (tolerance 1.00e-04)
Best objective -1.366666666667e+00, best bound -1.366666666667e+00, gap 0.0000%
Gurobi status= 2
<class 'numpy.ndarray'>
Matching added.
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-1185G7 @ 3.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 104 rows, 221 columns and 20262 nonzeros
Model fingerprint: 0x92f2e7b2
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [4e-01, 1e+00]
Presolv

KeyboardInterrupt: 

Exception ignored in: 'gurobipy._core.logcallbackstub'
Traceback (most recent call last):
  File "c:\Users\tdemeule\AppData\Local\anaconda3\Lib\site-packages\ipykernel\iostream.py", line 624, in write
    def write(self, string: str) -> Optional[int]:  # type:ignore[override]

KeyboardInterrupt: 



CPU model: 11th Gen Intel(R) Core(TM) i7-1185G7 @ 3.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 139 rows, 104 columns and 1606 nonzeros
Model fingerprint: 0x9deda815
Variable types: 1 continuous, 103 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+00]
  Objective range  [3e-02, 2e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+00]
Presolve removed 94 rows and 67 columns
Presolve time: 0.00s
Presolved: 45 rows, 37 columns, 376 nonzeros
Variable types: 0 continuous, 37 integer (37 binary)
Found heuristic solution: objective -1.1000000
Found heuristic solution: objective -1.1333333

Root relaxation: objective -1.673333e+00, 21 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   -1.67333  

In [ ]:
print(q.assignment)
print(MyModel.Xdecomp)
print(MyModel.Xdecomp_coeff)

## Run initial IP on generated data
Specify the number of students and schools, and run the models for this data.

In [ ]:
# Generate random data
parameters = DataGenParam(mean_pref = 6, capacity_ratio = 1) # Default parameters, except for mean_pref and capacity_ratio
#MyData = generate_data(n_students=400, n_schools=20, parameters = parameters, name="Test_DataGen", print_data=False, seed = 15)
MyData = generate_data(n_students=5, n_schools=4, parameters = parameters, name="Test_DataGen", print_data=False, seed = 0)

In [ ]:
# Print data if desired
print(MyData)

In [ ]:
# Generate the assignment from DA with Single Tie-Breaking with n_iter samples
n_iter = 1000
A = DA_STB(MyData, n_iter, 0, True)
print(A.assignment)

In [ ]:
# Solve the formulations
    # 'IMPR_RANK' refers to minimizing the expected rank while ensuring ex-post stability
    # 'STABLE' refers to maximizing the fraction of STABLE matchings in the decomposition
MyModel = Model(MyData, A, True)
q = MyModel.Solve("IMPR_RANK", "GUROBI", True)
#q = MyModel.Solve("STABLE", "GUROBI", True)

In [ ]:
# Print the solution
MyModel.print_solution()

In [ ]:
# Asses and visualize the difference
diff = Assignment(MyData, q.assignment - A.assignment, "TestDataGen_Diff")
diff.visualize()

## Heuristic subset weakly stable matchings
Runs an LP that minimizes the average rank while stochastically dominating the DA assignment with single-tie breaking, and while only using matchings that were used to compute DA probabilities. This is a heuristic.

In [ ]:
# Generate random data
parameters = DataGenParam(mean_pref = 5, capacity_ratio = 1) # Default parameters, except for mean_pref and capacity_ratio
#MyData = generate_data(n_students=400, n_schools=20, parameters = parameters, name="Test_DataGen", print_data=False, seed = 15)
MyData = generate_data(n_students=10, n_schools=4, parameters = parameters, name="500_25", print_data=False, seed = 1)

In [ ]:
# Print data if desired
print(MyData)

In [ ]:
# Generate the assignment from DA with Single Tie-Breaking with n_iter samples
n_iter = 1000
A = DA_STB(MyData, n_iter, 0, True)
print(A.assignment)

In [ ]:
# Solve the formulations
    # 'IMPR_RANK' refers to minimizing the expected rank while ensuring ex-post stability
MyModel = ModelHeuristicLP(MyData, A, True)
q = MyModel.Solve("IMPR_RANK", "GUROBI", True)
#q = MyModel.Solve("STABLE", "GUROBI", True)

In [ ]:
# Print the solution
MyModel.print_solution()

In [ ]:
print(np.max(q.assignment-A.assignment))

In [ ]:
# Asses and visualize the difference
diff = Assignment(MyData, q.assignment - A.assignment, "40_12_Diff")
diff.visualize()

## Heuristic Fractional stable random matching

In [ ]:
# Solve the formulations
    # 'IMPR_RANK' refers to minimizing the expected rank while ensuring ex-post stability
MyModelFS = ModelFracStable(MyData, A, True)
q = MyModelFS.Solve("IMPR_RANK", "GUROBI", True)
#q = MyModel.Solve("STABLE", "GUROBI", True)

In [ ]:
# FInd decomposition over weakly stable matchings of this fractionally stable matching (if decomposition exists)
MyModelCHECK = Model(MyData, q, True)
q_check = MyModelCHECK.Solve("STABLE", "GUROBI", True)

## Run code manual data
Manually enter data.

In [ ]:
# Define preferences of the students
# 'pref[i][k]' contains the position of the k-th ranked school in the preferences.
# We assume the preferences to be strict
# Note that preferences can be strict. We indicate this by a tuple () in the list.

# Example paper
n_stud = 4
n_schools = 4

file_name = "Ex_paper"

# Preferences students
pref = [['1', '3', '4', '2'],
       ['1','4','3','2'],
       # ['1', '4'],
       ['2','3', '4', '1'],
       ['2', '4', '3', '1']]

# Priorities schools
prior = [[('A', 'B'), 'C', 'D'],
        [('C', 'D'), 'A', 'B'],
        ['B', 'D', ('A', 'C')],
        ['A', 'C', ('B', 'D')]]


# Capacities schools
cap = [1,1,1,1]

# Names of students and schools
ID_stud = ["A", "B", "C", "D"]
ID_school = ["1", "2", "3", "4"]

# Also create the random matching upon which we want to improve
p = np.zeros(shape=(n_stud, n_schools))
p[0][0] = 1/2
p[1][0] = 1/2
p[2][1] = 1/2
p[3][1] = 1/2
p[0][2] = 3/8
p[2][2] = 3/8
p[1][3] = 3/8
p[3][3] = 3/8
p[0][3] = 1/8
p[2][3] = 1/8
p[1][2] = 1/8
p[3][2] = 1/8

In [ ]:
# Generate a data instance (and print it)
MyData = Data(n_stud, n_schools, pref, prior, cap, ID_stud, ID_school, file_name)
print(MyData)

In [ ]:
# Generate an Assignment instance (and visualize it)
A = Assignment(MyData, p, "Ex_paper")

# To visualize assignment
A.visualize()

In [ ]:
# Solve the formulations
    # 'IMPR_RANK' refers to minimizing the expected rank while ensuring ex-post stability
    # 'STABLE' refers to maximizing the fraction of STABLE matchings in the decomposition
MyModel = Model(MyData, A, False)
q = MyModel.Solve("IMPR_RANK", "GUROBI", False)
#q = MyModel.Solve("STABLE", "GUROBI", True)

In [ ]:
# Print the solution
MyModel.print_solution()

In [ ]:
# Asses and visualize the difference
diff = Assignment(MyData, q.assignment - p, "Ex_paper_Diff")
diff.visualize()

In [ ]:
tuple([1,2])